In [1]:
import sys
import os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

os.environ['CUDA_VISIBLE_DEVICES'] = '0'

In [2]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [3]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')


print(train_df.head())


                                               image                   id  \
0  ../../data/Train/CameraRGB/episode_0002_000287...  episode_0002_000287   
1  ../../data/Train/CameraRGB/episode_0008_000112...  episode_0008_000112   
2                 ../../data/Train/CameraRGB/804.png                  804   
3  ../../data/Train/CameraRGB/episode_0008_000286...  episode_0008_000286   
4  ../../data/Train/CameraRGB/episode_0003_000261...  episode_0003_000261   

                                               label  
0  ../../data/Train/CameraSeg/episode_0002_000287...  
1  ../../data/Train/CameraSeg/episode_0008_000112...  
2                 ../../data/Train/CameraSeg/804.png  
3  ../../data/Train/CameraSeg/episode_0008_000286...  
4  ../../data/Train/CameraSeg/episode_0003_000261...  


In [4]:
from models.segnet import model_segnetVGG16

model = model_segnetVGG16(3, image_shape=(320, 416, 3))


/home/faisal/anaconda3/envs/ai/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [5]:
from gen.datagen import oversample_generator_from_df, balanced_generator_from_df

BATCH_SIZE = 16
model_dir = '../../saved_models/segnet/segnet_v3/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

train_gen = oversample_generator_from_df(train_df, BATCH_SIZE, (320, 416), samples=2000)
valid_gen = balanced_generator_from_df(valid_df, BATCH_SIZE, (320, 416))

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy', 'mse'])


In [7]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model, 
                   train_gen, 
                   valid_gen, 
                   training_size=2000, 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=500,
                   gpus = 1)

Instructions for updating:
Use the retry module or similar alternatives.
Epoch 1/500
125/125 [==============================] - 142s 1s/step - loss: 0.6086 - acc: 0.8448 - mean_squared_error: 0.1076 - val_loss: 0.4471 - val_acc: 0.9298 - val_mean_squared_error: 0.0701

Epoch 00001: val_loss improved from inf to 0.44714, saving model to ../../saved_models/segnet/segnet_v3//model.hdf5
Epoch 2/500
125/125 [==============================] - 82s 657ms/step - loss: 0.3976 - acc: 0.9312 - mean_squared_error: 0.0600 - val_loss: 0.8672 - val_acc: 0.7693 - val_mean_squared_error: 0.1347

Epoch 00002: val_loss did not improve
Epoch 3/500
125/125 [==============================] - 82s 659ms/step - loss: 0.2970 - acc: 0.9566 - mean_squared_error: 0.0395 - val_loss: 0.3007 - val_acc: 0.9608 - val_mean_squared_error: 0.0377

Epoch 00003: val_loss improved from 0.44714 to 0.30066, saving model to ../../saved_models/segnet/segnet_v3//model.hdf5
Epoch 4/500
125/125 [==============================] - 82s


Epoch 00027: val_loss improved from 0.03986 to 0.03641, saving model to ../../saved_models/segnet/segnet_v3//model.hdf5
Epoch 28/500
125/125 [==============================] - 82s 653ms/step - loss: 0.0329 - acc: 0.9871 - mean_squared_error: 0.0057 - val_loss: 0.0351 - val_acc: 0.9904 - val_mean_squared_error: 0.0048

Epoch 00029: val_loss improved from 0.03641 to 0.03509, saving model to ../../saved_models/segnet/segnet_v3//model.hdf5
Epoch 30/500
125/125 [==============================] - 82s 658ms/step - loss: 0.0310 - acc: 0.9876 - mean_squared_error: 0.0055 - val_loss: 0.0366 - val_acc: 0.9899 - val_mean_squared_error: 0.0051

Epoch 00030: val_loss did not improve
Epoch 31/500
125/125 [==============================] - 82s 658ms/step - loss: 0.0293 - acc: 0.9882 - mean_squared_error: 0.0053 - val_loss: 0.0342 - val_acc: 0.9904 - val_mean_squared_error: 0.0048

Epoch 00031: val_loss improved from 0.03509 to 0.03418, saving model to ../../saved_models/segnet/segnet_v3//model.hdf5
E

KeyboardInterrupt: 

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [8]:
model.load_weights('../../saved_models/segnet/segnet_v3//model.hdf5')
model.save('../../saved_models/segnet/segnet_v3/model_saved.h5')